In [ ]:
#⚠️⚠️CAUTION: This code needs to run only once to setup the Results file

import pandas as pd
import numpy as np

df = pd.read_excel('./New_results.xlsx') # Change file name to Old_results.xlsx for old abstracts
df = df.dropna(axis = 0, ignore_index=True)
df['Sapling'] = ''
df.to_excel("Results_New.xlsx") # Change file name to Results_Old.xlsx for old abstracts results

In [ ]:
#function to extract data from AI detection site

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
import numpy as np;
import time
import re

driver = webdriver.Chrome()
driver.get("https://sapling.ai/ai-content-detector")

elem = driver.find_element(By.ID, "content-editor")

df = pd.read_excel('./Results.xlsx')
df = df.dropna(axis = 0 , ignore_index=False , subset= 'Abstract')

# HTML tags removal from abstract
def remove_html_tags(text):
    clean = re.compile('<.*?>')  # Compile the regex pattern for HTML tags
    return re.sub(clean, '', text)

df['Abstract'] = df["Abstract"].apply(remove_html_tags)

# Main function
def ai_text_score(text):
    elem.clear()
    elem.send_keys(text)
    time.sleep(5)
    res = driver.find_element(By.ID, "fake-prob")
    error = driver.find_element(By.ID , 'p-error')
    while res.text =="" or res.text == "...": # if results are still loading
        if error.is_displayed(): # if some error is encountered, the driver will reopen the browser
            driver.close()
            driver.get("https://sapling.ai/ai-content-detector")
        time.sleep(5)
        res = driver.find_element(By.ID, "fake-prob")
    return res.text

# Last index till which the function should run
index = 50

for ind in df.index:
    if pd.isnull(df['Sapling'][ind]):
        df['Sapling'][ind]= ai_text_score(df['Abstract'][ind])
    
    if ind == index:
        break

driver.close()
df.to_excel("Results.xlsx", index = False)